In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F, Row
from pyspark.sql.window import Window
from pyspark.sql.types import NumericType
from graphframes import GraphFrame
import re

# --- 1. Spark с Iceberg + MinIO ---
spark = SparkSession.getActiveSession()
if spark is not None:
    print("✅ Подключен к существующей SparkSession")
else:
    print("⚠️ Активная сессия не найдена — создаю новую...")
    spark = (
        SparkSession.builder
            .appName("glazkov_type_rest_graph_frame")
            .getOrCreate()
    )
# --- Подключаем PostgreSQL ---
# pg_url = "jdbc:postgresql://192.168.150.10:5432/MZ_DWH"
# pg_props = {
#     "user": "glazole",
#     "password": "MEbt7xr39yRhXUBj",
#     "driver": "org.postgresql.Driver"
# }

⚠️ Активная сессия не найдена — создаю новую...


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/07 06:36:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Create a Vertex DataFrame with unique ID column "id"
v = spark.createDataFrame([
    ("a", "Alice", 34),
    ("b", "Bob", 36),
    ("c", "Charlie", 30),
], ["id", "name", "age"])

# Create an Edge DataFrame with "src" and "dst" columns
e = spark.createDataFrame([
    ("a", "b", "friend"),
    ("b", "c", "follow"),
    ("c", "b", "follow"),
], ["src", "dst", "relationship"])
# Create a GraphFrame
g = GraphFrame(v, e)

# Query: Get in-degree of each vertex.
g.inDegrees.show()

# Query: Count the number of "follow" connections in the graph.
g.edges.filter("relationship = 'follow'").count()

# Run PageRank algorithm, and show results.
results = g.pageRank(resetProbability=0.01, maxIter=20)
results.vertices.select("id", "pagerank").show()

+---+--------+
| id|inDegree|
+---+--------+
|  b|       2|
|  c|       1|
+---+--------+



+---+------------------+
| id|          pagerank|
+---+------------------+
|  a|              0.01|
|  b|1.0905890109440908|
|  c|1.8994109890559092|
+---+------------------+



In [4]:
e.show(truncate=False)

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|a  |b  |friend      |
|b  |c  |follow      |
|c  |b  |follow      |
+---+---+------------+



In [5]:
spark.stop()